In [ ]:
# !pip install pandas openpyxl fuzzywuzzy python-Levenshtein requests


In [ ]:
import pandas as pd
import os

# Load the Excel file
input_path = "/content/South Carolina County Population 2020-2024 (DA).xlsx"
xls = pd.ExcelFile(input_path)

# Show all sheet names
sheet_names = xls.sheet_names
sheet_names


['Beaufort County Permits',
 'Chester County Permits',
 'Edgefield County Permits',
 'Greenville county permits',
 'Horry county permits',
 'Kershaw County Permits',
 'oconee county permits',
 'Pickens county permits ']

In [ ]:
import pandas as pd
import os
import re
import requests
from fuzzywuzzy import fuzz

# --- CONFIGURATION ---
input_path = "/content/South Carolina County Population 2020-2024 (DA).xlsx"
MAPBOX_API_KEY = "pk.eyJ1IjoicGFkbWFkZXkiLCJhIjoiY21iYWZ2bGVwMDBvMDJqc291NXNyY3Q1ZiJ9.VKNF16mUNwdOqcr7LJ7RZg"
output_dir = "/content/sheets_output"
os.makedirs(output_dir, exist_ok=True)

# --- FUZZY LOGIC FUNCTION ---
def standardize_permit_type(value):
    val = str(value).lower()
    if any(fuzz.partial_ratio(val, kw.lower()) > 80 for kw in ["Residential", "Family", "Home"]):
        return "Residential"
    elif any(fuzz.partial_ratio(val, kw.lower()) > 80 for kw in ["Commercial", "Com"]):
        return "Commercial"
    elif fuzz.partial_ratio(val, "Building") > 80:
        # Building can mean both, but if not caught earlier, default to Commercial
        return "Commercial"
    else:
        return value  # untouched if no match

# --- GEOCODING FUNCTION ---
def get_lat_lon(address):
    try:
        response = requests.get(
            f"https://api.mapbox.com/geocoding/v5/mapbox.places/{requests.utils.quote(address)}.json",
            params={"access_token": MAPBOX_API_KEY}
        )
        data = response.json()
        coords = data['features'][0]['center']
        return coords[1], coords[0]  # latitude, longitude
    except Exception:
        return None, None

# --- PROCESSING EACH SHEET ---
excel_file = pd.ExcelFile(input_path)
csv_files = []

for sheet_name in excel_file.sheet_names:
    df = excel_file.parse(sheet_name)

    # Standardize 'permit_type'
    df['permit_type'] = df['permit_type'].apply(standardize_permit_type)

    # Create 'full_address'
    df['full_address'] = df[['street_address', 'county', 'state', 'country']].astype(str).agg(', '.join, axis=1)

    # Get lat/lon
    df[['latitude', 'longitude']] = df['full_address'].apply(lambda x: pd.Series(get_lat_lon(x)))

    # Create 'lat-lon' column
    df['lat-lon'] = df[['latitude', 'longitude']].astype(str).agg(','.join, axis=1)

    # Export to CSV
    csv_path = os.path.join(output_dir, f"{sheet_name.replace('/', '_')}.csv")
    df.to_csv(csv_path, index=False)
    csv_files.append(csv_path)

# --- APPEND ALL CSVs INTO ONE ---
# combined_df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
# combined_csv_path = os.path.join(output_dir, "combined_output.csv")
# combined_df.to_csv(combined_csv_path, index=False)

print("All sheets processed and exported successfully.")


All sheets processed and exported successfully.


In [ ]:
# !pip install pandas openpyxl requests geojson


In [ ]:
import pandas as pd
import requests
import geojson

# Constants
input_path = "raw_data\Dentists.xlsx"
# output_csv_path = "sheets_output\Dentists_Geo.csv"
output_geojson_path = "sheets_output\Dentists_Geo.geojson"
MAPBOX_API_KEY = ""

# Step 1: Load Excel
df = pd.read_excel(input_path)

# Step 2: Normalize state abbreviations
state_mapping = {
    "SC": "South Carolina",
    "NC": "North Carolina",
    "GA": "Georgia"
}
df["state"] = df["state"].replace(state_mapping)

# Step 3: Clean street_address and build full_address
df["street_address"] = df["street_address"].str.replace("#", "", regex=False).str.strip()

df["full_address"] = (
    df["street_address"].astype(str) + ", " +
    df["city"].astype(str) + ", " +
    df["state"].astype(str) + " " +
    df["zip"].astype(str) + ", " +
    df["country"].astype(str)
)

# Step 4: Geocoding function using Mapbox
def geocode_address(address):
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{address}.json"
    params = {
        "access_token": MAPBOX_API_KEY,
        "limit": 1
    }
    response = requests.get(url, params=params)
    data = response.json()

    if data.get("features"):
        coords = data["features"][0]["geometry"]["coordinates"]
        return coords[1], coords[0]  # latitude, longitude
    return None, None

# Step 5: Apply geocoding
df["latitude"], df["longitude"] = zip(*df["full_address"].apply(geocode_address))

# Step 6: Create lat-lon column
df["lat-lon"] = df["latitude"].astype(str) + "," + df["longitude"].astype(str)

# Step 7: Export to CSV
# df.to_csv(output_csv_path, index=False)

# Step 8: Create GeoJSON
features = []
for _, row in df.iterrows():
    if pd.notnull(row["latitude"]) and pd.notnull(row["longitude"]):
        point = geojson.Point((row["longitude"], row["latitude"]))
        properties = {
            "business_name": row["business_name"],
            "full_address": row["full_address"],
            "lat-lon": row["lat-lon"]
        }
        features.append(geojson.Feature(geometry=point, properties=properties))

feature_collection = geojson.FeatureCollection(features)

with open(output_geojson_path, "w") as f:
    geojson.dump(feature_collection, f, indent=2)

print("Full process complete: Cleaned, normalized, geocoded, and exported CSV + GeoJSON.")


Full process complete: Cleaned, normalized, geocoded, and exported CSV + GeoJSON.
